In [27]:
import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.filterwarnings('ignore')

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

from sklearn.preprocessing import quantile_transform

%matplotlib inline

In [29]:
full = pd.read_csv('data/gen_1_holes.csv', index_col = 0)
#full = pd.read_csv('data/CA1_6_1_clean.csv', index_col = 0)

In [30]:
full.head()

,0N,1N,2N,3N,4N,5N,6N,7N,8N,9N,...,193N,194N,195N,196N,197N,198N,199N,x,y,ang
0,6.137285,6.097568,7.981179,2.657686,8.919547,4.274941,5.179740,3.329182,1.932031,0.178958,...,0.052298,1.043834,0.758416,0.041510,0.570631,0.361448,0.879365,0.316915,0.037568,-1.947778
1,1.128934,6.372495,10.931732,5.148409,4.566580,7.450268,3.145072,2.717946,1.464976,2.153725,...,0.177679,1.146309,0.179111,1.535442,0.826470,0.002301,0.435258,0.397302,0.084844,-1.813300
2,0.853555,3.371220,9.012780,3.539809,6.774548,10.779202,2.431839,1.256614,2.002973,0.709518,...,0.098760,1.443941,0.464352,0.682519,0.280986,0.235136,0.557921,0.449147,0.079035,-1.691014
3,0.699315,5.615830,5.725598,2.757512,3.292234,13.453683,6.159488,2.802111,1.799720,1.586054,...,0.213535,0.937239,0.383083,0.716624,0.692223,0.140585,0.121586,0.474667,0.057570,-1.627992
4,2.934969,0.965006,2.585714,4.674477,6.684643,6.955064,10.598396,7.053686,1.640949,5.961785,...,0.171356,1.847330,0.162991,0.130131,1.095337,1.971624,1.846373,0.540623,0.028656,-1.484823


In [32]:
full = full.drop(['x','y', 'ang'], axis=1)
#full = full.drop(['x','y', 'segment'], axis=1)

In [5]:
mi_best_neurons = pd.read_csv('data/mi_best_neurons.csv', index_col = 0)
worst = mi_best_neurons[-573:]

In [33]:
full = full.iloc[:60]

In [6]:
full.drop(worst.index, axis=1, inplace=True)

# 250ms cycle

In [7]:
full_theta = full[:0]
for i in range(0, full.shape[0], 5):
    full_theta = full_theta.append(full[i:i+5].mean(), ignore_index=True)

In [8]:
full_theta.head()

,30N,70N,76N,105N,118N,119N,128N,132N,146N,153N,157N,230N,273N,381N,382N,396N,461N
0,0.002683,-0.002224,-0.016238,-0.010134,-0.015599,-0.006800,-0.014974,-0.002927,-0.014776,-0.017407,-0.020017,-0.010256,-0.002332,0.012720,-0.011935,0.001049,0.005601
1,0.003893,-0.003432,-0.017445,-0.010457,-0.015755,-0.007718,-0.016295,-0.003065,-0.014243,-0.014855,-0.020949,-0.005078,-0.001545,0.010516,-0.012704,0.000927,0.005029
2,0.001497,-0.004334,-0.014896,-0.010736,-0.014939,-0.009164,-0.018426,-0.000749,-0.014394,-0.018882,-0.020746,-0.002286,0.000052,0.009041,-0.010464,0.000512,0.003970
3,-0.001186,-0.002356,-0.015547,-0.010579,-0.015827,-0.006787,-0.020459,-0.002044,-0.013006,-0.022289,-0.018011,-0.006708,-0.004518,0.007465,-0.012132,0.001713,0.004494
4,-0.004451,-0.000775,-0.015726,-0.011102,-0.015925,-0.004175,-0.022074,-0.003972,-0.014024,-0.026286,-0.015889,-0.010812,-0.006187,0.007111,-0.015100,0.004146,0.004706


In [9]:
full.shape

(6119, 17)

In [80]:
import json

with open('data/lr_neurons.json') as f:
    neurons = json.load(f)

In [ ]:
full = full.loc[:, neurons]

In [ ]:
#mis = pd.read_csv('data/gen_mi_best_neurons.csv', index_col=0);
#mis['neuron'] = mis.index

In [35]:
from dionysus import Filtration, homology_persistence, init_diagrams, Simplex
from collections import Counter
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def get_cycles_for_filtration(filtration, end_time=np.inf):
    m = homology_persistence(filtration, prime=2, progress=True)
    dgms = init_diagrams(m, filtration)
    cycles = []
    for i, dgm in enumerate(dgms):
        for pt in dgm:
            cycles.append((i, pt.birth, pt.death))
            #print((i, pt.birth,pt.death))
    cycles = pd.DataFrame(cycles, columns=['dimension', 'birth_time', 'death_time'])
    cycles['living_time'] = cycles['death_time'] - cycles['birth_time']
    inf_life = (cycles['living_time'] == np.inf)
    cycles['living_time'][inf_life] = end_time - cycles['birth_time'][inf_life]
    return cycles.sort_values('dimension', ascending=True)


def homologies(df, number_of_ap):
    # time - simplexes that live > time%
    # number_of_ap = (time/100) * len(df)
    #number_of_ap=90
    #print(number_of_ap)

    df = quantile_transform(df)
    filtration = Filtration()
    for alpha in np.arange(0.50, 0.99, 0.05):
        print(alpha)
        neuron_activity = df > alpha
        sims = Counter()
        for row in neuron_activity:
            vert = tuple(np.where(row)[0])
            sims[vert]+=1
            
        complexes = []
        for s, cnt in sims.items():
            if s == ():
                continue
            if cnt > number_of_ap:
                complexes.append(s)
        
        
        all_simplexes = set()
        for compl in complexes:
            for s in list(powerset(compl))[1:]:
                all_simplexes.add(s)
                
        all_simplexes = sorted(list(all_simplexes), key=lambda x: len(x))
        for sim in all_simplexes:
            filtration.append(Simplex(sim, alpha))

    if len(filtration) == 0:
        return None
    cycles = get_cycles_for_filtration(filtration, 0.95)
    cycles.columns = [f"{col}_{number_of_ap/df.shape[0]}" for col in cycles.columns]
    
    return cycles.reset_index(drop=True)

In [ ]:
for n in [0, 60, 70, 80, 90, 100, 200, 500][:1]:
    print(n)
    #mi_best_neurons = mis[:10]['neuron']
    #data = full.drop(['x', 'y'], axis=1)[mi_best_neurons.index]
    df = homologies(full.iloc[:, :], n)
    if df is None:
        print("no simplexes in filtration")
    else:
        print(df.T)
        print()

In [ ]:
filtration = Filtration()

In [ ]:
len(filtration)

In [ ]:
s1 = Simplex([0, 1], 0.1)
s2 = Simplex([0, 2], 0.1)
s3 = Simplex([1, 2], 0.1)
filtration.append(s1)
filtration.append(s2)
filtration.append(s3)
filtration.append(Simplex([0], 0.1))
filtration.append(Simplex([1], 0.1))
filtration.append(Simplex([2], 0.1))

# s1 = Simplex([0, 1], 0.2)
# s2 = Simplex([0, 2], 0.2)
# s3 = Simplex([1, 2], 0.2)
# s4 = Simplex([0, 4], 0.2)
# filtration.append(s1)
# filtration.append(s2)
# filtration.append(s3)
# filtration.append(s4)

In [ ]:
filtration

In [ ]:
end_time = np.inf
m = homology_persistence(filtration, prime=2, progress=True)
dgms = init_diagrams(m, filtration)
cycles = []
for i, dgm in enumerate(dgms):
    for pt in dgm:
        cycles.append((i, pt.birth, pt.death))
        #print((i, pt.birth,pt.death))
cycles = pd.DataFrame(cycles, columns=['dimension', 'birth_time', 'death_time'])
cycles['living_time'] = cycles['death_time'] - cycles['birth_time']
inf_life = (cycles['living_time'] == np.inf)
cycles['living_time'][inf_life] = end_time - cycles['birth_time'][inf_life]

cycles.sort_values('dimension', ascending=True)